In [ ]:
from bs4 import BeautifulSoup
from collections import namedtuple
from datetime import datetime
import json
import logging
import os
import requests
import time
from subprocess import Popen, PIPE

try:
    from subprocess import DEVNULL
except ImportError:
    import os
    DEVNULL = open(os.devnull, 'wb')

logging.basicConfig(
    format='%(asctime)s %(levelname)-8s %(message)s',
    level=logging.INFO,
    datefmt='%Y-%m-%d %H:%M:%S')

# Global Variables

In [ ]:
initial_dir = os.getcwd()

Set `test_folders` to `None` if no longer testing.

In [ ]:
test_folders = [
    'en/develop/tutorials/articles/01-introduction-to-liferay-development'
]

# Git CLI

In [ ]:
def _git(*args, stderr=PIPE):
    cmd = ['git'] + list(args)
    
    if args[0] != 'config':
        logging.info(' '.join(cmd))
    
    pipe = Popen(cmd, stdout=PIPE, stderr=stderr)
    out, err = pipe.communicate()

    return out.decode('UTF-8', 'replace')

# Named Tuples

In [ ]:
GitHubRepository = namedtuple(
    'GitHubRepository',
    ' '.join(['git_root', 'origin', 'upstream', 'branch', 'project_folder'])
)

In [ ]:
CrowdInRepository = namedtuple(
    'CrowdinRepository',
    ' '.join(['project_id', 'api_key'])
)

In [ ]:
TranslationRepository = namedtuple(
    'TranslationRepository',
    ' '.join(['github', 'crowdin'])
)

In [ ]:
def get_repository(git_root, branch, project_folder, project_id):
    os.chdir(git_root)

    origin_url = _git('remote', 'get-url', 'origin').strip()
    origin = origin_url.split(':')[1][:-4]

    upstream_url = _git('remote', 'get-url', 'upstream').strip()
    upstream = upstream_url.split(':')[1][:-4]
    
    api_key = _git('config', 'crowdin.api-key.%s' % project_id).strip()
    
    os.chdir(initial_dir)
    
    return TranslationRepository(
        GitHubRepository(git_root, origin, upstream, branch, project_folder),
        CrowdInRepository(project_id, api_key)
    )

In [ ]:
repositories = []

for branch in ['7.1.x']:
    repositories.append(
        get_repository(
            '/home/minhchau/Work/liferay/liferay-docs', branch,
            'en/develop/tutorials/articles',
            'liferay-documentation'))

# GitHub API Utilities

Given an OAuth token, a base URL, and an API path, we make requests against GitHub by using the `requests` module, and we interpret the response as JSON.

In [ ]:
github_oauth_token = _git('config', 'github.oauth-token').strip()
assert(github_oauth_token is not None)

github_base_url = 'https://api.github.com'

def github_request(api_path, request_type=None, data=None):
    headers = {
        'user-agent': 'python',
        'authorization': 'token %s' % github_oauth_token,
        'accept': 'application/vnd.github.inertia-preview+json'
    }
    
    if data is None:
        r = requests.get(github_base_url + api_path, headers=headers)
    elif request_type == 'PATCH':
        r = requests.patch(github_base_url + api_path, json=data, headers=headers)
    elif request_type == 'POST':
        r = requests.post(github_base_url + api_path, json=data, headers=headers)

    if r.status_code < 200 or r.status_code >= 400:
        logging.error('HTTP Error: %d' % r.status_code)
        return (r.status_code, r.headers, None)

    return (r.status_code, r.headers, r.json())

In addition to the general privacy restrictions, the GitHub API also adds a rate limit to the REST API endpoints. We'll want to make sure that any request we send to GitHub is aware of those limitations.

In [ ]:
global remaining

remaining = 0

def wait_for_rate_limit_reset():
    global remaining

    if remaining > 0:
        remaining -= 1
        return

    while True:
        status_code, headers, result = github_request('/rate_limit')
        
        resources = result['resources']['core']
        remaining = resources['remaining']

        if remaining > 0:
            remaining -= 1
            return

        wait_time = 1 + int(resources['reset'] - datetime.now().timestamp())

        logging.error('Waiting %d seconds for rate limit reset' % wait_time)

        time.sleep(wait_time)

In addition to waiting for the rate limit to reset, we also need to think about what happens if we make multiple requests against the API.

For example, let's say we had an hourly script that routinely crawled all the forks of the `liferay-portal` repository checking for new pull requests or new pull request comments. When we do this, we'll also want to make sure that when we make a request to GitHub, that we exclude data that overlaps with data we've already retrieved. This filtering method both waits for the rate limit, and then filters the results, after making the request.

In [ ]:
def filter_github_request(api_path, request_type=None, data=None, field_name=None, min_field_value=None):
    global remaining

    if remaining == 0:
        wait_for_rate_limit_reset()

    status_code, headers, result = github_request(api_path, request_type, data)
    
    if result is None:
        remaining = int(headers['X-RateLimit-Remaining']) if 'X-RateLimit-Remaining' in headers else 0

        if remaining == 0:
            wait_for_rate_limit_reset()
            status_code, headers, result = github_request(api_path, request_type, data)

    if result is None:
        return (status_code, headers, [])

    if min_field_value is None:
        return (status_code, headers, result)

    return (status_code, headers, [item for item in result if item[field_name] > min_field_value])

In [ ]:
def filter_github_request_all(api_path, request_type=None, data=None, field_name=None, min_field_value=None):
    results = []
    
    while api_path is not None:
        status_code, headers, new_results = filter_github_request(
            api_path, request_type, data, field_name, min_field_value)

        results.extend(new_results)
        
        lower_headers = {
            key.lower(): value for key, value in headers.items()
        }

        if 'link' not in lower_headers:
            break
        
        api_path = None

        for link in lower_headers['link'].split(','):
            url_info, rel_info = [info.strip() for info in link.split(';')]

            rel = rel_info.split('"')[1]
            print(rel)
            if rel == 'next':
                api_path = url_info[len('<https://api.github.com'):-1]

    return status_code, headers, results

Next, we'll want to make sure that our OAuth token actually works and can visit the GitHub repositories we've defined.

In [ ]:
def is_repository_accessible(reviewer_url):
    logging.info('github check-repo-access %s' % reviewer_url)

    api_path = '/repos/%s' % reviewer_url

    status_code, headers, result = filter_github_request(api_path)
    
    return result is not None

In [ ]:
for git_repository, crowdin_repository in repositories:
    assert(is_repository_accessible(git_repository.origin))
    assert(is_repository_accessible(git_repository.upstream))

# CrowdIn CLI

In [ ]:
def _crowdin(*args, stderr=PIPE):
    cmd = ['crowdin'] + list(args)
    
    logging.info(' '.join(cmd))
    
    pipe = Popen(cmd, stdout=PIPE, stderr=stderr)
    out, err = pipe.communicate()

    return out.decode('UTF-8', 'replace').strip()

Use `pandoc` to disable word wrapping to improve machine translations.

In [ ]:
def _pandoc(filename, *args):
    with open(filename, 'r') as f:
        lines = f.readlines()
        
        title_pos = -1
        toc_pos = -1

        for i, line in enumerate(lines):
            if title_pos == -1 and line.find('#') == 0:
                title_pos = i
            elif line.find('[TOC') == 0:
                toc_pos = i
        
        head_lines = ''.join(lines[0:max(title_pos, toc_pos)+1])
        tail_lines = ''.join(lines[max(title_pos, toc_pos)+1:])

    cmd = ['pandoc'] + list(args)

    pipe = Popen(cmd, stdin=PIPE, stdout=PIPE, stderr=PIPE)
    out, err = pipe.communicate(input=tail_lines.encode('UTF-8'))
    
    nowrap_lines = out.decode('UTF-8', 'replace')
    
    with open(filename, 'w') as f:
        f.write(head_lines)
        f.write('\n')
        f.write(nowrap_lines)

Generate a `crowdin.yaml` file to tell the CLI what to do.

In [ ]:
def configure_crowdin(repository, files):
    files_json = json.dumps([{
        'source': file + '/*.markdown' if os.path.isdir(file) else file,
        'translation': '%two_letters_code%/' + file[3:] + '/%original_file_name%'
    } for file in files], indent=2)

    with open('%s/crowdin.yaml' % repository.github.git_root, 'w') as f:
        f.write('''
"project_identifier" : "{crowdin_project_id}"
"api_key" : "{crowdin_api_key}"
"base_path" : "{git_root}"
"preserve_hierarchy": true

"files": {files}
'''.format(
        crowdin_project_id=repository.crowdin.project_id,
        crowdin_api_key=repository.crowdin.api_key,
        git_root=repository.github.git_root,
        files=files_json
    ))

Wrapper functions to upload sources and download translations.

In [ ]:
def crowdin_upload_sources(repository, new_files):
    for file in new_files:
        _pandoc(file, '--from=gfm', '--to=gfm', '--wrap=none')
    
    if len(new_files) > 0:
        configure_crowdin(repository, new_files)
        _crowdin('upload', 'sources', '-b', repository.github.branch)

    _git('reset', '--hard')
        
    return get_crowdin_file_info(repository)

In [ ]:
def crowdin_download_translations(repository, all_files):
    configure_crowdin(repository, get_folders(all_files))
    
    _crowdin('download', '-l', 'ja', '-b', repository.github.branch)

# CrowdIn API Utilities

In [ ]:
crowdin_base_url = 'https://api.crowdin.com/api/project/'

def crowdin_request(repository, api_path, request_type='GET', data=None, files=None):
    headers = {
        'user-agent': 'python'
    }
    
    request_url = crowdin_base_url + repository.crowdin.project_id + api_path + \
        '?key=' + repository.crowdin.api_key
    
    if request_type == 'GET':
        r = requests.get(request_url, data=data, headers=headers)
    else:
        r = requests.post(request_url, data=data, files=files, headers=headers)

    if r.status_code < 200 or r.status_code >= 400:
        logging.error('HTTP Error: %d' % r.status_code)
        return (r.status_code, None)

    return (r.status_code, r.text)

# Crowdin File Management API

API calls to download the translation memory and glossary.

In [ ]:
def save_translation_memory(repository):
    logging.info('crowdin-api download-tm')

    data = {
        'source_language': 'en',
        'target_language': 'ja'
    }
    
    status_code, response_text = crowdin_request(repository, '/download-tm', 'GET', data)

    if response_text is not None:
        with open('%s/%s.tmx' % (initial_dir, repository.crowdin.project_id), 'w') as f:
            f.write(response_text)

In [ ]:
def save_glossary(repository):
    logging.info('crowdin-api download-glossary')

    status_code, response_text = crowdin_request(repository, '/download-glossary', 'GET')

    if response_text is not None:
        with open('%s/%s.tbx' % (initial_dir, repository.crowdin.project_id), 'w') as f:
            f.write(response_text)

API call to delete existing translation.

In [ ]:
def delete_translation(repository, file):
    logging.info('crowdin-api delete-file %s' % file)
    
    data = {
        'file': file[len(repository.github.branch)+1:],
        'branch': repository.github.branch
    }
    
    return crowdin_request(repository, '/delete-file', 'POST', data)

API calls to find the file IDs for CrowdIn (used to generate links within the issues).

In [ ]:
def extract_crowdin_file_info(files_element, current_path, file_info):
    for item in files_element.children:
        if item.name != 'item':
            continue
        
        item_name = item.find('name').text
        item_node_type = item.find('node_type').text

        item_path = current_path + '/' + item_name if current_path is not None else item_name

        file_info[item_path] = {
            'phrases': int(item.find('phrases').text),
            'approved': int(item.find('approved').text)
        }
        
        if item_node_type == 'file':
            file_info[item_path]['id'] = item.find('id').text
        else:
            extract_crowdin_file_info(item.find('files'), item_path, file_info)

In [ ]:
def get_crowdin_file_info(repository):
    logging.info('crowdin-api language-status')

    data = {
        'language': 'ja'
    }
    
    status_code, response_text = crowdin_request(
        repository, '/language-status', 'POST', data)
    
    file_info = {}

    if response_text is not None:
        soup = BeautifulSoup(response_text)
        extract_crowdin_file_info(soup.find('files'), None, file_info)

    return file_info

# Crowdin Translation API

Send requests to CrowdIn to do automated translation (translation memory, then machine translation, then translation memory again).

In [ ]:
def translate_with_memory(repository, files):
    logging.info('crowdin-api pre-translate tm -b %s' % repository.github.branch)

    data = {
        'languages[]': ['ja'],
        'files[]': files,
        'method': 'tm',
        'auto_approve_option': 0,
        'import_duplicates': 1,
        'apply_untranslated_strings_only': 1,
        'perfect_match': 0,
        'json': 1
    }
    
    return crowdin_request(repository, '/pre-translate', 'POST', data)

In [ ]:
def translate_with_machine(repository, files):
    logging.info('crowdin-api pre-translate mt -b %s' % repository.github.branch)

    data = {
        'languages[]': ['ja'],
        'files[]': files,
        'method': 'mt',
        'engine': 'google',
        'json': 1
    }

    return crowdin_request(repository, '/pre-translate', 'POST', data)

In [ ]:
def pre_translate(repository, translation_needed, file_info):
    translation_files = set()
    
    for file in translation_needed:
        prefix = '%s/%s' % (repository.github.branch, file)
        translation_files.update([key for key in file_info.keys() if key.find(prefix) == 0])
    
    if len(translation_files) == 0:
        return

    translate_with_memory(repository, translation_files)
    translate_with_machine(repository, translation_files)
    translate_with_memory(repository, translation_files)

    crowdin_download_translations(repository, translation_files)

# Markdown File Tracking

Some utility methods to convert a folder to a list of files and a list of files into a list of folders.

In [ ]:
def get_files(folder):
    files = []
    
    for name in os.listdir(folder):
        path = '%s/%s' % (folder, name)

        if os.path.isdir(path):
            files.extend(get_files(path))
        else:
            files.append(path)
    
    return list(files)

In [ ]:
def get_folders(files):
    return sorted(set([os.path.dirname(file) if os.path.isfile(file) else file for file in files]))

We only need to worry about translating folders with `.markdown` files.

In [ ]:
def is_translation_eligible(file, language_id):
    if file[0:2] != language_id or file[2] != '/':
        return False

    if file[-9:] == '.markdown' or file[-3:] == '.md':
        return True
    
    return False

In [ ]:
def get_md_files(output, language_id):
    if test_folders is not None:
        test_files = set()
        
        for test_folder in set(test_folders):
            test_files.update(get_files(test_folder))
    
        return sorted([file for file in test_files if is_translation_eligible(file, language_id)])
    
    files = output.split('\n')
    
    get_folders()

    return [file for file in files if is_translation_eligible(file, language_id)]

# GitHub Issue API

Retrieve milestones.

In [ ]:
def get_milestone_number(repository, milestone_map, milestone_title):
    if milestone_title in milestone_map:
        return milestone_map[milestone_title]
    
    api_path = '/repos/%s/milestones' % repository.github.origin

    data = {
        'title': milestone_title
    }
    
    status_code, headers, milestone = filter_github_request(api_path, 'POST', data)
    
    milestone_map[milestone_title] = milestone['number']
    
    return milestone['number']

In [ ]:
def get_milestone_map(repository):
    milestone_map_file = '%s/%s/milestones.json' % (initial_dir, repository.github.origin)
    milestone_map = {}

    if os.path.exists(milestone_map_file):
        with open(milestone_map_file, 'r') as f:
            milestone_map = json.load(f)
    else:
        api_path = '/repos/%s/milestones?state=all' % repository.github.origin
    
        status_code, headers, milestones = filter_github_request(api_path)
        
        milestone_map = {
            milestone['title']: milestone['number']
                for milestone in milestones
        }
        
    get_milestone_number(repository, milestone_map, 'human')
    get_milestone_number(repository, milestone_map, 'machine')

    with open(milestone_map_file, 'w') as f:
        json.dump(milestone_map, f)

    return milestone_map

Local cache letting us know how folders are mapped to issues.

In [ ]:
def get_issue_map(repository):
    issue_map_file = '%s/%s/issues.json' % (initial_dir, repository.github.origin)
    issue_map = {}

    if os.path.exists(issue_map_file):
        with open(issue_map_file, 'r') as f:
            issue_map = json.load(f)

    return issue_map

In [ ]:
def save_issue_map(repository, issue_map):
    issue_map_file = '%s/%s/issues.json' % (initial_dir, repository.github.origin)

    with open(issue_map_file, 'w') as f:
        json.dump(issue_map, f)

Consistent way of generating issue bodies from metadata.

In [ ]:
def get_issue_line(repository, prefix, key, metadata):
    translation_completion = int(100 * metadata['approved'] / metadata['phrases'])
    
    if 'id' not in metadata:
        return '\n**%s (%s%%)**\n' % \
            (key[prefix.rfind('/')+1:], translation_completion)
    else:
        return '* [%s](https://crowdin.com/translate/%s/%s/en-ja) (%s%%)' % \
            (key[key.rfind('/')+1:], repository.crowdin.project_id, metadata['id'], translation_completion)

In [ ]:
def get_issue_body(repository, folder, file_info):
    prefix = '%s/%s' % (repository.github.branch, folder)

    matching_files = sorted([
        (key, metadata) for key, metadata in file_info.items() if key.find(prefix) == 0
    ])
    
    if len(matching_files) == 0:
        return ''

    external_links = [
        get_issue_line(repository, prefix, key, metadata)
            for key, metadata in matching_files
    ]
    
    return '\n'.join(external_links).strip()

Make sure that issues always exist.

In [ ]:
def init_issue(repository, folder, file_info):
    issue_map = get_issue_map(repository)
    
    if repository.github.branch not in issue_map:
        issue_map[repository.github.branch] = {}
    
    issues = issue_map[repository.github.branch]
    
    if folder in issues:
        return False, issues[folder]

    logging.info('github create-issue')
    
    data = {
        'title': '%s - %s' % (repository.github.branch, folder),
        'body': get_issue_body(repository, folder, file_info),
        'milestone': get_milestone_map(repository)['machine']
    }
    
    api_path = '/repos/%s/issues' % repository.github.origin
    status_code, headers, result = filter_github_request(api_path, 'POST', data)
    
    issues[folder] = result['number']

    save_issue_map(repository, issue_map)

    return True, issues[folder]

In [ ]:
def init_issues(repository, all_files, file_info):
    all_folders = get_folders(all_files)
    new_translation_needed = []

    for folder in all_folders:
        new_issue, issue_number = init_issue(repository, folder, file_info)
        
        if new_issue:
            new_translation_needed.append(folder)

    return new_translation_needed

Reopen the issue on update.

In [ ]:
def reopen_issue(repository, folder, file_info):
    new_issue, issue_number = init_issue(repository, folder, file_info)

    api_path = '/repos/%s/issues/%d' % (repository.github.origin, issue_number)
    status_code, headers, result = filter_github_request(api_path)
    
    milestone = result['milestone'] if 'milestone' in result else None
    milestone_number = milestone['number'] if milestone is not None else None

    issue_body = get_issue_body(repository, folder, file_info)

    if result['state'] != 'closed' and result['body'] == issue_body and milestone_number is not None:
        return new_issue, issue_number

    data = {
        'state': 'open',
        'body': issue_body
    }
    
    if milestone_number is None:
        data['milestone'] = get_milestone_map(repository)['machine']

    status_code, headers, result = filter_github_request(api_path, 'PATCH', data)
    
    return new_issue, issue_number

In [ ]:
def reopen_issues(repository, new_files, file_info):
    new_folders = get_folders(new_files)
    new_translation_needed = list(new_files)
    
    for folder in new_folders:
        new_issue, issue_number = reopen_issue(repository, folder, file_info)
        
        if new_issue:
            new_translation_needed = [
                file for file in new_translation_needed
                    if file != folder and file.find(folder + '/') != 0
            ]

            new_translation_needed.append(folder)
    
    return new_translation_needed

Close a GitHub issue when translation is completed.

In [ ]:
def close_issue(repository, folder, file_info):
    new_issue, issue_number = init_issue(repository, folder, file_info)

    api_path = '/repos/%s/issues/%d' % (repository.github.origin, issue_number)

    data = {
        'state': 'closed'
    }

    status_code, headers, result = filter_github_request(api_path, 'PATCH', data)
    
    return new_issue, issue_number

# GitHub Project API

Retrieve the current list of repository projects.

In [ ]:
def get_github_project_map(origin):
    status_code, headers, projects = filter_github_request('/repos/%s/projects' % origin)
    
    project_map = {}
    
    for project in projects:
        project_name = project['name']
        
        separator_pos = project_name.find(' - ')
        
        branch = project_name[:separator_pos]
        folder = project_name[separator_pos+3:]
        
        if branch not in project_map:
            project_map[branch] = {}
        
        project_map[branch][folder] = project['id']
    
    return project_map

Create an empty project if one does not already exist for the folder.

In [ ]:
def init_github_project(project_map, repository):
    for project_folder, project_number in project_map[repository.github.branch].items():
        if project_folder == repository.github.project_folder:
            return project_number

    api_path = '/repos/%s/projects' % repository.github.origin

    data = {
        'name': '%s - %s' % (repository.github.branch, repository.github.project_folder)
    }

    status_code, headers, project = filter_github_request(api_path, 'POST', data)
    project_map[repository.github.branch][repository.github.project_folder] = project['id']
    
    return project['id']

Create the proper columns ("To do", "In progress", "Done") for the project.

In [ ]:
def init_github_column(columns, project_number, column_index, column_name):
    for column in columns:
        if column_name == column['name']:
            return column
    
    api_path = '/projects/%s/columns' % project_number
    
    data = {
        'name': column_name
    }

    status_code, headers, column = filter_github_request(api_path, 'POST', data)

    if len(columns) != column_index:
        api_path = '/projects/columns/%s/moves' % column['id']
        
        data = {
            'position': 'first' if column_index == 0 else 'after:%d' % (column_index - 1)
        }
    
    columns.insert(column_index, column)
    
    return column

In [ ]:
def init_github_columns(column_map, project_number):
    if project_number in column_map:
        columns = column_map[project_number]
    else:
        status_code, headers, columns = filter_github_request('/projects/%s/columns' % project_number)
        column_map[project_number] = columns
    
    init_github_column(columns, project_number, 0, 'To do')
    init_github_column(columns, project_number, 1, 'In progress')
    init_github_column(columns, project_number, 2, 'Done')
    
    return columns

Make sure that a card exists for the specified GitHub issue.

In [ ]:
def get_github_card(card_map, repository, project_number, issue_number, target_column_index, columns):
    if project_number not in card_map:
        card_map[project_number] = {}
        
        for column in columns:
            column_id = column['id']
            column_name = column['name']

            status_code, headers, cards = filter_github_request('/projects/columns/%s/cards' % column_id)

            card_map[project_number].update({
                int(card['content_url'][card['content_url'].rfind('/')+1:]): {
                    'card_id': card['id'], 'column_id': column_id
                } for card in cards if 'content_url' in card
            })

    if issue_number in card_map[project_number]:
        return card_map[project_number][issue_number]

    api_path = '/repos/%s/issues/%d' % (repository.github.origin, issue_number)
    status_code, headers, issue = filter_github_request(api_path)

    column_index = target_column_index if target_column_index is not None else 0
    
    api_path = '/projects/columns/%s/cards' % columns[column_index]['id']

    data = {
        'content_id': issue['id'],
        'content_type': 'Issue'
    }

    status_code, headers, card = filter_github_request(api_path, 'POST', data)

    card_map[project_number][issue_number] = {
        'card_id': card['id'], 'column_id': columns[column_index]['id']
    }

    return card

Make sure the card is in the right column.

In [ ]:
def update_issue_column(project_map, column_map, card_map, repository, folder, issue_number, target_column_index):
    project_number = init_github_project(project_map, repository)
    columns = init_github_columns(column_map, project_number)
    target_column_id = None if target_column_index is None else columns[target_column_index]['id']
    
    card = get_github_card(card_map, repository, project_number, issue_number, target_column_index, columns)

    if target_column_id is None or card['column_id'] == target_column_id:
        return

    api_path = '/projects/columns/cards/%s/moves' % card['card_id']
    
    data = {
        'position': 'top',
        'column_id': target_column_id
    }

    status_code, headers, update_info = filter_github_request(api_path, 'POST', data)

In [ ]:
def update_issue_columns(repository, file_info, target_column_map):
    project_map = get_github_project_map(repository.github.origin)
    column_map = {}
    card_map = {}

    for folder, target_column_index in target_column_map.items():
        init_issue(repository, folder, file_info)
        issue_map = get_issue_map(repository)
        issue_number = issue_map[repository.github.branch][folder]

        update_issue_column(
            project_map, column_map, card_map, repository,
            folder, issue_number, target_column_index)

In [ ]:
def update_translation_issues(repository, file_info):
    issues = []

    api_path = '/repos/%s/issues?milestone=%d' % \
        (repository.github.origin, get_milestone_map(repository)['human'])

    status_code, headers, issues = filter_github_request_all(api_path)
    
    target_column_map = {}
    
    for issue in issues:
        branch, folder = issue['title'].split(' - ')
        
        if branch == repository.github.branch and folder.find(repository.github.project_folder) == 0:
            key = '%s/%s' % (branch, folder)
            
            if key in file_info:
                approved = file_info[key]['approved']
                phrases = file_info[key]['phrases']

                if approved == 0:
                    target_column_map[folder] = 0
                elif approved == phrases:
                    target_column_map[folder] = 2
                else:
                    target_column_map[folder] = 1
            else:
                target_column_map[folder] = None
    
    update_issue_columns(repository, file_info, target_column_map)

# Cron job

Retrieve the list of all files we should translate and all updated files that we need to re-translated.

In [ ]:
def get_branch_files(branch):
    _git('fetch', 'upstream', '--no-tags', '%s:refs/remotes/upstream/%s' % (branch, branch))

    diff_output = _git('diff', '--name-only', '%s..upstream/%s' % (branch, branch))
    new_files = get_md_files(diff_output, 'en')

    lstree_output = _git('ls-tree', '-r', '--name-only', branch)
    all_files = get_md_files(lstree_output, 'en')
    
    return new_files, all_files

Keep CrowdIn and GitHub translations in sync.

In [ ]:
def update_sources(repository, new_files):
    _git('checkout', repository.github.branch)
    _git('merge', 'upstream/%s' % repository.github.branch)

    return crowdin_upload_sources(repository, new_files)

In [ ]:
def update_translations(repository, all_files):
    now = datetime.now()

    crowdin_download_translations(repository, all_files)

    status_output = '\n'.join([line[3:] for line in _git('status', '--porcelain').split('\n')])
    commit_files = get_md_files(status_output, 'ja')

    if len(commit_files) == 0:
        return
    
    for file in commit_files:
        if file[-3:] == '.md':
            continue

        md_file = file[:-9] + '.md'

        if os.path.isfile(md_file):
            os.remove(md_file)
            os.rename(file, md_file)

    status_output = '\n'.join([line[3:] for line in _git('status', '--porcelain').split('\n')])
    commit_files = get_md_files(status_output, 'ja')

    if len(commit_files) == 0:
        return    
    
    _git('add', *commit_files)
    _git('commit', '-m', 'Updated translations %s' % now.strftime("%Y-%m-%d %H:%M:%S"))

Perform any needed GitHub issue maintenance to keep project boards usable, perform any needed CrowdIn file maintenance to stay under the quota.

In [ ]:
def cleanup_files(repository, file_info):
    to_close = set()
    to_delete = set()
    
    prefix = repository.github.branch + '/'
    issue_map = get_issue_map(repository)
    
    for file, metadata in file_info.items():
        if file.find(prefix) != 0:
            continue
        
        folder = file[len(prefix):]
        
        if folder not in issue_map[repository.github.branch]:
            continue
        
        if metadata['phrases'] == metadata['approved']:
            to_close.add(file)

    for folder in to_close:
        prefix = '%s/%s/' % (repository.github.branch, folder)

        for file, metadata in file_info.items():
            if file.find(prefix) == 0:
                to_delete.add(file)

    for folder in to_close:
        close_issue(repository, folder, file_info)
            
    for file in to_delete:
        print(file)
        #del file_info[file]
        #delete_translation(repository, file)

Update local copies of translations, translation memory, and glossaries.

In [ ]:
for repository in repositories:
    logging.info('cd %s' % repository.github.git_root)
    os.chdir(repository.github.git_root)

    new_files, all_files = get_branch_files(repository.github.branch)
    file_info = update_sources(repository, new_files)

    new_translation_needed = []

    new_translation_needed.extend(init_issues(repository, all_files, file_info))
    new_translation_needed.extend(reopen_issues(repository, new_files, file_info))

    if len(new_translation_needed) > 0:
        pre_translate(repository, new_translation_needed, file_info)

    update_translations(repository, all_files)
    cleanup_files(repository, file_info)
    update_translation_issues(repository, file_info)

    save_translation_memory(repository)
    save_glossary(repository)

    logging.info('cd -')
    os.chdir(initial_dir)